In [1]:
cd

C:\Users\56961


In [2]:
cd Desktop\portafolio

C:\Users\56961\Desktop\portafolio


In [3]:
get_ipython().magic('reset -sf')

C:\Users\56961\AppData\Local\Temp\ipykernel_13184\3674724974.py:1: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


In [4]:

# cargar librerias

# definir librerias
import random
import numpy as np
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import networkx as nx
import pickle 
from collections import Counter
import csv

import networkx.algorithms.isomorphism as iso

import scipy.io as sio
import numpy as np
from datetime import datetime

import seaborn as sns
import pandas as pd
import numpy as np
import io
import requests
import warnings
warnings.filterwarnings('ignore', '.*do not.*', )
warnings.warn('DelftStack')
warnings.warn('Do not show this message')
import warnings
warnings.filterwarnings('ignore')

# cd :  usar cd para fijar y ver la direccion en la que se esta.

#from librerias  import algoritmos_sample as alg_sample
#from librerias  import algoritmos_generales as alg_gral
#from librerias  import algoritmos_freq as alg_freq
#from librerias  import alg_fabrica_grafo as alg_grafo
#from IPython import get_ipython

C:\Users\56961\AppData\Local\Temp\ipykernel_13184\103248778.py:30: UserWarning: DelftStack
  warnings.warn('DelftStack')


In [5]:
from librerias import algoritmos_infectar as alg_infec
from librerias  import alg_sample_v2 as alg_sample
from librerias  import algoritmos_frecuentista as alg_freq

In [26]:
# funcion definitiva}
def funcion_experimentacion(G,name_df,num_graph,tipo_graph, L_metodo_infec, L_porc_nodos_infec, L_tipo_sample_p_gral, L_porc_nodos_muestra, L_n_metodo_est):
    
    
    set_nodos=set(G.nodes())
    t=len(set_nodos)
    cant_nodos=t
    dict_nei_G={x_i:set(G.neighbors(x_i))  for x_i in list(G.nodes())}    
    dic_grados_G=dict(G.degree( list(G.nodes()) ))
    
    list_nei_G=[]
    for x_i in list(G.nodes()):
        list_nei_G.append(set(G.neighbors(x_i)))
    prob_acept,cant_acept=0.5, 50
    prob_acept_p_inf,cant_acept_p_inf=0.5, 50
    
    col_num_graph=[]
    col_tipo_graph=[]
    col_metodo_infec=[]
    col_porc_nodos_infec=[]
    col_tipo_sample_p_gral=[]
    col_porc_nodos_muestra=[]
    col_tipo_sample_p_infec=[]
    col_porc_nodos_muestra_inf=[]
    col_n_metodo_est=[]

    col_Valor_Estimado=[]
    col_valor_error=[]
    col_valor_error_rel=[]    
    col_Tiempo_ejecucion=[]
    
    lista_infec_y_samplekey=[]
    lista_infec_y_sample=[]
    # Opciones variables
    #Inicio  for 
    iter=0
    for metodo_infec in L_metodo_infec:
        for porc_nodos_infec in L_porc_nodos_infec:
            # Opciones Fijas
            ## Infectar nodos:
            input_mpi=[0]*7
            input_mpi[0]=G
            input_mpi[1]=metodo_infec
            input_mpi[2]=porc_nodos_infec
            input_mpi[3]=list_nei_G
            input_mpi[4]=dic_grados_G
            input_mpi[5]=prob_acept_p_inf
            input_mpi[6]=cant_acept_p_inf
            lista_infectados=alg_infec.manager_proceso_infec(input_mpi)
            set_infectados=set(lista_infectados)
            set_noinfectados=set(G.nodes())- set_infectados
            
            for tipo_muestreo in L_tipo_sample_p_gral:
                for porc_nodos in L_porc_nodos_muestra:
                    
                    date_1 = datetime.now()

                    # Muestreo de poblacion general
                    input_sm=[0]*7
                    input_sm[0]=G
                    input_sm[1]=tipo_muestreo
                    input_sm[2]=porc_nodos
                    input_sm[3]=list_nei_G
                    input_sm[4]=dic_grados_G
                    input_sm[5]=prob_acept
                    input_sm[6]=cant_acept
                    output_sample_manager =alg_sample.sample_manager(input_sm)
                    if tipo_muestreo == "2" or tipo_muestreo == "6":
                        print("tipo_muestreo",tipo_muestreo)
                        Muestra_general, est_2E,cant_aprox_hop_need = output_sample_manager[0],output_sample_manager[1],output_sample_manager[2]
                    else:
                        Muestra_general=output_sample_manager
                    
                    input_base_2=[t,set_nodos,dict_nei_G,dic_grados_G,porc_nodos_infec,set_infectados]
                    input_base=alg_freq.calculos_basicos(tipo_muestreo,Muestra_general,input_base_2) 
                    
                    input_base_2=[dict_nei_G,dic_grados_G,set_infectados]
                    
                    for n_metodo_est in L_n_metodo_est:  
                                iter=iter+1
                                print(iter,metodo_infec,porc_nodos_infec,tipo_muestreo, porc_nodos, n_metodo_est)                        

                                if n_metodo_est>=3:
                                    if tipo_muestreo=="1" or tipo_muestreo=="2" or tipo_muestreo=="3" or tipo_muestreo=="4":
                                        continue
                                
                                if n_metodo_est>=10 and tipo_muestreo=="5":
                                    continue
                                if n_metodo_est>=3 and (metodo_infec=="5" or metodo_infec=="6"):
                                    continue                                    
                                        
                                Estimacion_alg,metodo_est=alg_freq.alg_estimador_manager_zip(n_metodo_est,input_base,input_base_2,Muestra_general)
                                #Estimacion_alg=1
                                date_2 = datetime.now()

                                #Estimacion_alg
                                error=len(lista_infectados) - Estimacion_alg
                                Tiempo_ejecucion=date_2 - date_1

                                # guardar informacion de la iteracion en listas correspondientes
                                col_num_graph.append(num_graph)
                                col_tipo_graph.append(tipo_graph)
                                
                                col_metodo_infec.append(metodo_infec)
                                col_porc_nodos_infec.append(porc_nodos_infec)
                                
                                tipo_sample_p_gral,porc_nodos_muestra=tipo_muestreo,porc_nodos
                                col_tipo_sample_p_gral.append(tipo_sample_p_gral)
                                col_porc_nodos_muestra.append(porc_nodos_muestra)
                                
                                col_n_metodo_est.append(metodo_est)

                                col_Valor_Estimado.append(Estimacion_alg)
                                col_valor_error.append(error)
                                col_valor_error_rel.append( error/ len(lista_infectados))
                                col_Tiempo_ejecucion.append(Tiempo_ejecucion)
                                
    # Termino for 

    # Generar DF
    n_col1="Id_grafo"
    n_col2="Tipo_grafo"
    n_col3="Metodo_infeccion"
    n_col4="Porc_infectados"
    n_col5="Tipo_sample_p_gral"
    n_col6="Porc_nodos_muestra"
    
    
    n_col9="Num_metodo_est"  

    n_col10="Valor_Estimado"
    n_col11="Valor_error(Exac - Est)"
    n_col12="Valor_error_rel(Exac - Est)"
    n_col13="Tiempo_ejecucion"    


    col1=col_num_graph
    col2=col_tipo_graph
    col3=col_metodo_infec
    col4=col_porc_nodos_infec

    col5=col_tipo_sample_p_gral
    col6=col_porc_nodos_muestra
    
    
    col9=col_n_metodo_est

    col10=col_Valor_Estimado
    col11=col_valor_error
    col12=col_valor_error_rel
    col13=col_Tiempo_ejecucion    

    dic_df={n_col1:col1 ,n_col2:col2 ,n_col3:col3 ,n_col4:col4 ,n_col5:col5 ,n_col6:col6,n_col9:col9,n_col10:col10,n_col11:col11,n_col12:col12,n_col13:col13}            
    Df_est_m1=pd.DataFrame(dic_df)

    # Guardar DF con nombre adecuado
    name_save=name_df

    var_save= Df_est_m1 # variable a guardar
    # creamos fichero binario 
    fichero_bin= open( name_save, "wb")
    # metemos la var_wsave en el fichero binario
    pickle.dump(var_save, fichero_bin)
    fichero_bin.close()
    

    
    return Df_est_m1

# Todas las configuraciones:

In [27]:
# input comple
L_metodo_infec=["1","2","3"]
L_porc_nodos_infec=[0.01,0.1,0.5, 0.9]

L_tipo_sample_p_gral=["1","2","3"]
L_porc_nodos_muestra=[0.1,0.5,0.9]

L_n_metodo_est=[0,1,2]

# Confiración en especifico

In [28]:
# input comple
L_metodo_infec=["5"]
L_porc_nodos_infec=[0.1]

L_tipo_sample_p_gral=["5"]
L_porc_nodos_muestra=[0.1]
L_n_metodo_est=[1]

Caso: Escala libre

In [29]:
n=6000
#lista_valor_m=[int(n/2) , int(n/8)]
lista_valor_m=[int(n/8)]
tipo_graph="Escala"

#for num_graph in range(1,25):
for num_graph in range(1):    
    for m in lista_valor_m:
        date_1=datetime.now()
        
        name_est_last= "Datos_grafos/grafos_v2/"+"graph_escala"+"n"+str(6000)+"m"+str(m)+"n°"+str(num_graph)
        name_df= "Experimentacion/database_exp/"+"graph_escala"+"n"+str(6000)+"m"+str(m)+"n°"+str(num_graph)

        file = open(name_est_last, 'rb')
        # dump information to that filea
        G= pickle.load(file)
        #close the file
        file.close()

        Df_est_m1=funcion_experimentacion(G,name_df,num_graph,tipo_graph, L_metodo_infec, L_porc_nodos_infec, L_tipo_sample_p_gral, L_porc_nodos_muestra, L_n_metodo_est)
        #print(Df_est_m1.head(3))
        date_2=datetime.now()    
        print(date_2-date_1)
Df_est_m1.head(2)

1 5 0.1 5 0.1 1
0:00:36.542818


,Id_grafo,Tipo_grafo,Metodo_infeccion,Porc_infectados,Tipo_sample_p_gral,Porc_nodos_muestra,Num_metodo_est,Valor_Estimado,Valor_error(Exac - Est),Valor_error_rel(Exac - Est),Tiempo_ejecucion
0,0,Escala,5,0.1,5,0.1,PIMPLE,713.316062,-113.316062,-0.18886,0 days 00:00:00.048022


Caso: Grafo aleatorio

In [31]:
#lista_valor_p=[1,0.5 ,0.2]
lista_valor_p=[0.2]
tipo_graph="Aleatorio"


#for num_graph in range(79,100):    
for num_graph in range(1):        
    for p in lista_valor_p:
        date_1=datetime.now()
        
        name_est_last= "Datos_grafos/grafos_v2/"+"graph"+"n"+str(6000)+"p"+str(1/p)+"n°"+str(num_graph)
        name_df= "Experimentacion/database_exp/"+"graph"+"n"+str(6000)+"p"+str(1/p)+"n°"+str(num_graph)
        if p == 0.5:
            name_est_last= "Datos_grafos/grafos_v2/"+"graph"+"n"+str(6000)+"prob"+str(1/p)+"n°"+str(num_graph)
            
        file = open(name_est_last, 'rb')
        # dump information to that filea
        G= pickle.load(file)
        #close the file
        file.close()

        Df_est_m1=funcion_experimentacion(G,name_df,num_graph,tipo_graph, L_metodo_infec, L_porc_nodos_infec, L_tipo_sample_p_gral, L_porc_nodos_muestra, L_n_metodo_est)
        #print(Df_est_m1.head(3))
        date_2=datetime.now()    
        print(date_2-date_1)
Df_est_m1

1 5 0.1 5 0.1 1
0:00:04.663743


,Id_grafo,Tipo_grafo,Metodo_infeccion,Porc_infectados,Tipo_sample_p_gral,Porc_nodos_muestra,Num_metodo_est,Valor_Estimado,Valor_error(Exac - Est),Valor_error_rel(Exac - Est),Tiempo_ejecucion
0,0,Aleatorio,5,0.1,5,0.1,PIMPLE,600.156097,-0.156097,-0.00026,0 days 00:00:00.040007
